In [6]:
#import library
import nltk
import string
import re
import pandas as pd
import numpy as np

#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer


In [7]:
#read data
def load_data():
    data = pd.read_csv('D:/Kuliah Online/Pemrosesan Teks Praktik/PTP Project/Data/G20Indonesia_Normalization.csv')
    return data

#Pembuatan dataframe
df_twit = load_data()

#menampilkan data teratas
df_twit.head(10)
#len(df_twit)

,teks
0,A dan C Jawaban nya min mainNakerpresidensiGGI...
1,Ayo siapa yang takut .. Amerika Rusia Ketemuan...
2,Gerakan nasional literasi digital Digital tale...
3,Indonesia ... IndonesiaJuara IndonesiaHebatInd...
4,Keren bangetMas Memtri dengan Program Merdeka ...
5,Konsumen konsumen Industri dan Rumah Tangga pe...
6,Lapor pak bisanya cewe sekarang bego bencong j...
7,Pak Yth kami Perencana Koperasi memastikan bah...
8,Sepertinya .. di Bali ... Saat ... GIndonesiaA...
9,cewe ga jelas bencong joget malah diteriakin m...


In [8]:
#definisi dataframe
df = pd.DataFrame(df_twit[['teks']])

#menampilkan dataframe
df.head(10)

,teks
0,A dan C Jawaban nya min mainNakerpresidensiGGI...
1,Ayo siapa yang takut .. Amerika Rusia Ketemuan...
2,Gerakan nasional literasi digital Digital tale...
3,Indonesia ... IndonesiaJuara IndonesiaHebatInd...
4,Keren bangetMas Memtri dengan Program Merdeka ...
5,Konsumen konsumen Industri dan Rumah Tangga pe...
6,Lapor pak bisanya cewe sekarang bego bencong j...
7,Pak Yth kami Perencana Koperasi memastikan bah...
8,Sepertinya .. di Bali ... Saat ... GIndonesiaA...
9,cewe ga jelas bencong joget malah diteriakin m...


In [9]:
#menghilangkan mention/user
def remove_pattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r:
        tweet = re.sub(i, '', tweet)
    return tweet    
df['remove_user'] = np.vectorize(remove_pattern)(df['teks'], "@[\w]*")

In [10]:

def tweet_clean(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'RT :[\s]+', '', tweet)
    
     # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
     # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #Happy Emoticons
    emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
    
    #Sad Emoticons
    emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
    #all emoticons (happy + sad)
    emoticons = emoticons_happy.union(emoticons_sad)
              
    # TOKENIZING
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweets_clean = []    
    for word in tweet_tokens:
        # FILTERING
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            # STEMMING
            stem_word = stemmer.stem(word) #stemming word
            tweets_clean.append(stem_word)
    return tweets_clean
df['tweet_clean'] = df['remove_user'].apply(lambda x: tweet_clean(x))

In [11]:
df.head(10)

,teks,remove_user,tweet_clean
0,A dan C Jawaban nya min mainNakerpresidensiGGI...,A dan C Jawaban nya min mainNakerpresidensiGGI...,"[a, c, nya, min, mainnakerpresidensiggindonesi..."
1,Ayo siapa yang takut .. Amerika Rusia Ketemuan...,Ayo siapa yang takut .. Amerika Rusia Ketemuan...,"[ayo, takut, , amerika, rusia, ketemu, , bal, ..."
2,Gerakan nasional literasi digital Digital tale...,Gerakan nasional literasi digital Digital tale...,"[gera, nasional, literasi, digital, digital, t..."
3,Indonesia ... IndonesiaJuara IndonesiaHebatInd...,Indonesia ... IndonesiaJuara IndonesiaHebatInd...,"[indonesia, , indonesiajuara, indonesiahebatin..."
4,Keren bangetMas Memtri dengan Program Merdeka ...,Keren bangetMas Memtri dengan Program Merdeka ...,"[keren, bangetmas, memtri, program, merdeka, b..."
5,Konsumen konsumen Industri dan Rumah Tangga pe...,Konsumen konsumen Industri dan Rumah Tangga pe...,"[konsumen, konsumen, industri, rumah, tangga, ..."
6,Lapor pak bisanya cewe sekarang bego bencong j...,Lapor pak bisanya cewe sekarang bego bencong j...,"[lapor, bisa, cewe, bego, bencong, joget, suka..."
7,Pak Yth kami Perencana Koperasi memastikan bah...,Pak Yth kami Perencana Koperasi memastikan bah...,"[yth, rencana, koperasi, core, values, g]"
8,Sepertinya .. di Bali ... Saat ... GIndonesiaA...,Sepertinya .. di Bali ... Saat ... GIndonesiaA...,"[, bal, , , gindonesiaakan, putus]"
9,cewe ga jelas bencong joget malah diteriakin m...,cewe ga jelas bencong joget malah diteriakin m...,"[cewe, ga, bencong, joget, diteriakin, mending..."


In [12]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [13]:
df.head(10)

,teks,remove_user,tweet_clean,Tweet
0,A dan C Jawaban nya min mainNakerpresidensiGGI...,A dan C Jawaban nya min mainNakerpresidensiGGI...,"[a, c, nya, min, mainnakerpresidensiggindonesi...",a c nya min mainnakerpresidensiggindonesiaewgi...
1,Ayo siapa yang takut .. Amerika Rusia Ketemuan...,Ayo siapa yang takut .. Amerika Rusia Ketemuan...,"[ayo, takut, , amerika, rusia, ketemu, , bal, ...",ayo takut amerika rusia ketemu bal gindonesia ...
2,Gerakan nasional literasi digital Digital tale...,Gerakan nasional literasi digital Digital tale...,"[gera, nasional, literasi, digital, digital, t...",gera nasional literasi digital digital talent ...
3,Indonesia ... IndonesiaJuara IndonesiaHebatInd...,Indonesia ... IndonesiaJuara IndonesiaHebatInd...,"[indonesia, , indonesiajuara, indonesiahebatin...",indonesia indonesiajuara indonesiahebatindones...
4,Keren bangetMas Memtri dengan Program Merdeka ...,Keren bangetMas Memtri dengan Program Merdeka ...,"[keren, bangetmas, memtri, program, merdeka, b...",keren bangetmas memtri program merdeka belajar...
5,Konsumen konsumen Industri dan Rumah Tangga pe...,Konsumen konsumen Industri dan Rumah Tangga pe...,"[konsumen, konsumen, industri, rumah, tangga, ...",konsumen konsumen industri rumah tangga daya k...
6,Lapor pak bisanya cewe sekarang bego bencong j...,Lapor pak bisanya cewe sekarang bego bencong j...,"[lapor, bisa, cewe, bego, bencong, joget, suka...",lapor bisa cewe bego bencong joget suka mendin...
7,Pak Yth kami Perencana Koperasi memastikan bah...,Pak Yth kami Perencana Koperasi memastikan bah...,"[yth, rencana, koperasi, core, values, g]",yth rencana koperasi core values g
8,Sepertinya .. di Bali ... Saat ... GIndonesiaA...,Sepertinya .. di Bali ... Saat ... GIndonesiaA...,"[, bal, , , gindonesiaakan, putus]",bal gindonesiaakan putus
9,cewe ga jelas bencong joget malah diteriakin m...,cewe ga jelas bencong joget malah diteriakin m...,"[cewe, ga, bencong, joget, diteriakin, mending...",cewe ga bencong joget diteriakin mending doa p...


In [14]:
df.sort_values('Tweet', inplace = True)
df.drop(df.columns[[0,1]], axis = 1, inplace = True)
df.drop_duplicates(subset ='Tweet', keep = 'first', inplace = True)
df.to_csv('D:/Kuliah Online/Pemrosesan Teks Praktik/PTP Project/Data/G20Indonesia_After_Preprocessing.csv',encoding='utf8', index=False)
df.head(10)

,tweet_clean,Tweet
0,"[a, c, nya, min, mainnakerpresidensiggindonesi...",a c nya min mainnakerpresidensiggindonesiaewgi...
22,"[acara, jg, momentum, kuat, promosi, produk, u...",acara jg momentum kuat promosi produk umkm lok...
23,"[alhamdulillah, presiden, elon, musk, hadir, g...",alhamdulillah presiden elon musk hadir gindone...
24,"[alhamdulillah, presiden, hadir, november]",alhamdulillah presiden hadir november
14,"[amerika, negara, besara, negara, pro, damai, ...",amerika negara besara negara pro damai amerika...
25,"[anggota, g, seru, perang, mungkin hal, menkeu...",anggota g seru perang mungkin hal menkeusmi ko...
1,"[ayo, takut, , amerika, rusia, ketemu, , bal, ...",ayo takut amerika rusia ketemu bal gindonesia ...
102,"[bahas, isu, temu, sherpa, track, harap, hasil...",bahas isu temu sherpa track harap hasil politi...
28,"[bahlil, presidensi, g, indonesia, momentum, b...",bahlil presidensi g indonesia momentum baik ra...
8,"[, bal, , , gindonesiaakan, putus]",bal gindonesiaakan putus
